## Loading data

In [20]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-11-23 04:59:22--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.23.207, 74.125.203.207, 74.125.204.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.23.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  22.6MB/s    in 2.9s    

2024-11-23 04:59:26 (22.6 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Reading the Data into arrays

In [21]:
from PIL import Image
import numpy as np

cats_dir = os.path.join(train_dir + "/cats")
dogs_dir = os.path.join(train_dir + "/dogs")

i = 0
NUMBER_OF_EXAMPLES = 5

x_train = []
y_train = []

while i < NUMBER_OF_EXAMPLES:
  if i % 2 == 0:
    im = Image.open(os.path.join(cats_dir, os.listdir(cats_dir)[i])).convert("RGB")
    im_resized = im.resize((150, 150))
    x_train.append(np.array(im_resized))
    y_train.append(1)
  else:
    im = Image.open(os.path.join(dogs_dir, os.listdir(dogs_dir)[i])).convert("RGB")
    im_resized = im.resize((150, 150))
    x_train.append(np.array(im_resized))
    y_train.append(0)
  i += 1

x_train = np.array(x_train)
y_train = np.array(y_train)

## Beginning to define the model (this is where you come in, I loaded the pretrained model for you)

In [22]:
import tensorflow as tf

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)

# Train the model

In [23]:
from keras import layers, models

model = models.Sequential([
    pretrained_model,
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 44s 44s/step - accuracy: 0.6000 - loss: 0.6838
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 6.9319e-04
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 5.9700e-05
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 2.5263e-05
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 1.7141e-05
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 1.0000 - loss: 1.3077e-05
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 1.0261e-05
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 7.7980e-06
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 5.9128e-06
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 4.6599e-06


# Make Predictions

In [ ]:
import random
import matplotlib.pyplot as plt

# Randomly choose between cats and dogs
category = random.randrange(1, 3)

if category == 1:
  category = cats_dir
else:
  category = dogs_dir

# Choose a random image from a list of image files for category, build path
image_files = os.listdir(category)
random_image_filename = random.choice(image_files)
test_image_path = os.path.join(category, random_image_filename)

# Open the image, resize it, and normalize it (same preprocessing as training)
test_image = Image.open(test_image_path).convert("RGB")
test_image_resized = test_image.resize((150, 150))  # Resize to 150x150
test_image_array = np.array(test_image_resized) / 255.0  # Normalize the pixel values

# Expand the dimensions to match the input shape (batch size of 1)
test_image_array = np.expand_dims(test_image_array, axis=0)

# Make a prediction using the trained model
prediction = model.predict(test_image_array)
predicted_class = 'Cat' if prediction[0][0] > 0.5 else 'Dog'

# Display result
plt.imshow(test_image_resized)
plt.axis('off')
plt.title(f'Predicted class: {predicted_class}')
plt.show()